## RAG Pipelinefor Data ingetion to Vector DB

In [1]:
import os
from langchain_community.document_loaders import PyMuPDFLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

/opt/anaconda3/envs/rag_v2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

def process_documents(file_path):
    all_pages = []
    DIR = Path(file_path)

    pdf_files = list(DIR.glob("**/*.pdf"))
    print("Document Files Length:", len(pdf_files))

    for file in pdf_files:
        try:
            loader = PyMuPDFLoader(str(file))
            documents = loader.load()   # ← use plural consistently

            for page in documents:
                page.metadata["source"] = file.name
                page.metadata["file_type"] = "pdf"

            all_pages.extend(documents)
            print(f"Loaded {file.name}, pages: {len(documents)}")

        except Exception as e:
            print(f"❌ Error loading {file.name}: {e}")

    print("Total pages loaded:", len(all_pages))
    return all_pages
documents = process_documents("../data/SUBREN/PDF")

Document Files Length: 4492
Loaded SUBREN-1683.pdf, pages: 5
Loaded SUBREN-3094.pdf, pages: 4
Loaded SUBREN-3080.pdf, pages: 4
Loaded SUBREN-208.pdf, pages: 2
Loaded SUBREN-4089.pdf, pages: 2
Loaded SUBREN-220.pdf, pages: 2
Loaded SUBREN-3916.pdf, pages: 2
Loaded SUBREN-546.pdf, pages: 4
Loaded SUBREN-77.pdf, pages: 4
Loaded SUBREN-552.pdf, pages: 4
Loaded SUBREN-63.pdf, pages: 2
Loaded SUBREN-3902.pdf, pages: 6
Loaded SUBREN-234.pdf, pages: 2
Loaded SUBREN-1126.pdf, pages: 5
Loaded SUBREN-3731.pdf, pages: 3
Loaded SUBREN-2349.pdf, pages: 2
Loaded SUBREN-4738.pdf, pages: 9
Loaded SUBREN-3057.pdf, pages: 7
Loaded SUBREN-88.pdf, pages: 2
Loaded SUBREN-1898.pdf, pages: 2
Loaded SUBREN-1640.pdf, pages: 2
Loaded SUBREN-1654.pdf, pages: 2
Loaded SUBREN-3043.pdf, pages: 6
Loaded SUBREN-3725.pdf, pages: 4
Loaded SUBREN-1132.pdf, pages: 4
Loaded SUBREN-4062.pdf, pages: 7
Loaded SUBREN-2413.pdf, pages: 5
Loaded SUBREN-585.pdf, pages: 4
Loaded SUBREN-2375.pdf, pages: 4
Loaded SUBREN-4704.pdf, pag

In [3]:
# from pathlib import Path
# from langchain_community.document_loaders import UnstructuredWordDocumentLoader

# def process_documents(file_path):
#     all_pages = []
#     DIR = Path(file_path)

#     doc_files = list(DIR.glob("**/*.doc"))
#     print("Document Files Length:", len(doc_files))

#     for file in doc_files:
#         try:
#             loader = UnstructuredWordDocumentLoader(str(file))
#             documents = loader.load()

#             for page in documents:
#                 page.metadata["source"] = file.name
#                 page.metadata["file_type"] = "doc"

#             all_pages.extend(documents)
#             print(f"Loaded {file.name}, pages: {len(documents)}")

#         except Exception as e:
#             print(f"Error loading {file.name}: {e}")
#             continue

#     return all_pages

# all_subren_pages = process_documents("../data/SUBREN")


In [4]:
all_subren_pages=documents
all_subren_pages

[Document(metadata={'producer': 'LibreOffice 25.8.4.2 (AARCH64)', 'creator': '', 'creationdate': '2026-01-29T11:43:27+05:30', 'source': 'SUBREN-1683.pdf', 'file_path': '../data/SUBREN/PDF/SUBREN-1683.pdf', 'total_pages': 5, 'format': 'PDF 1.7', 'title': '[#SUBREN-1683] Salesforce Trigger for Credit Note Generation', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20260129114327+05'30'", 'page': 0, 'file_type': 'pdf'}, page_content='[SUBREN-1683]\xa0Salesforce Trigger for Credit Note Generation Created: 18/Oct/22 \nUpdated: 23/Dec/22 \xa0Resolved: 23/Dec/22 \nStatus:\nClosed\nProject:\nSubscription & Renewals\nComponent/s:\nNone \nAffects Version/s:\nNone \nFix Version/s:\n12.01.22 S&R Test 5 \nType: \nStory \nPriority: \nUrgent \nReporter: \nDillon Wright [X] (Inactive) \nAssignee: \nNitin Job \nResolution: \nDone \nVotes: \n0 \nLabels: \nA&E \nRemaining \nEstimate:\n16h \nTime Spent:\nNot Specified \nOriginal Estimate: 16h \

### Text Splitting

In [5]:
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs= text_splitter.split_documents(documents)
    print(f"Split {len(documents)} Documents into ", len(split_docs), " chunks.")

    if split_docs:
        print("example chunk : ")
        print("Sample Content: ", split_docs[0].page_content[:300], "...")
        print("Metadata: ", split_docs[0].metadata)

    return split_docs

all_subren_docs = split_documents(all_subren_pages)

Split 18367 Documents into  32180  chunks.
example chunk : 
Sample Content:  [SUBREN-1683] Salesforce Trigger for Credit Note Generation Created: 18/Oct/22 
Updated: 23/Dec/22  Resolved: 23/Dec/22 
Status:
Closed
Project:
Subscription & Renewals
Component/s:
None 
Affects Version/s:
None 
Fix Version/s:
12.01.22 S&R Test 5 
Type: 
Story 
Priority: 
Urgent 
Reporter: 
Dillon  ...
Metadata:  {'producer': 'LibreOffice 25.8.4.2 (AARCH64)', 'creator': '', 'creationdate': '2026-01-29T11:43:27+05:30', 'source': 'SUBREN-1683.pdf', 'file_path': '../data/SUBREN/PDF/SUBREN-1683.pdf', 'total_pages': 5, 'format': 'PDF 1.7', 'title': '[#SUBREN-1683] Salesforce Trigger for Credit Note Generation', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20260129114327+05'30'", 'page': 0, 'file_type': 'pdf'}


In [6]:
all_subren_docs

[Document(metadata={'producer': 'LibreOffice 25.8.4.2 (AARCH64)', 'creator': '', 'creationdate': '2026-01-29T11:43:27+05:30', 'source': 'SUBREN-1683.pdf', 'file_path': '../data/SUBREN/PDF/SUBREN-1683.pdf', 'total_pages': 5, 'format': 'PDF 1.7', 'title': '[#SUBREN-1683] Salesforce Trigger for Credit Note Generation', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20260129114327+05'30'", 'page': 0, 'file_type': 'pdf'}, page_content='[SUBREN-1683]\xa0Salesforce Trigger for Credit Note Generation Created: 18/Oct/22 \nUpdated: 23/Dec/22 \xa0Resolved: 23/Dec/22 \nStatus:\nClosed\nProject:\nSubscription & Renewals\nComponent/s:\nNone \nAffects Version/s:\nNone \nFix Version/s:\n12.01.22 S&R Test 5 \nType: \nStory \nPriority: \nUrgent \nReporter: \nDillon Wright [X] (Inactive) \nAssignee: \nNitin Job \nResolution: \nDone \nVotes: \n0 \nLabels: \nA&E \nRemaining \nEstimate:\n16h \nTime Spent:\nNot Specified \nOriginal Estimate: 16h \

### Embedding & VectorStore DB

In [7]:
import chromadb
from chromadb.config import Settings
import numpy as np
from sentence_transformers import SentenceTransformer
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity


In [8]:
class EmbeddingManager:
    def __init__(self,model_name : str ="all-MiniLM-L6-v2"):
        self.model_name = model_name #This model has 384 dimensions
        self.model= None
        self._load_model()

    def _load_model(self):
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully: {self.model.get_sentence_embedding_dimension()} dimensions")
        except Exception as e:
            print(f"Error loading model: {e}")

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        if not self.model:
            raise ValueError("Embedding model is not loaded.")
        embeddings = self.model.encode(texts, show_progress_bar=True, convert_to_numpy=True)
        print(f"Generated embeddings for {len(texts)} texts.")
        print(f"Embedding shape: {embeddings.shape}")
        return embeddings

In [9]:
eM= EmbeddingManager()
eM.model


Loading embedding model: all-MiniLM-L6-v2


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2130.46it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded successfully: 384 dimensions


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

## Vectore Store

In [26]:
class VectorStore:
    def __init__(self, collection_name: str = "Doc_data", persist_directory: str = "../data/vector_store"):
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()
 

    def _initialize_store(self):
        try:
            print("Initializing ChromaDB client...")
            os.makedirs(self.persist_directory, exist_ok=True)
            
            import chromadb
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            print("ChromaDB client initialized.")

            # Use create_collection instead of get_or_create_collection
            if self.collection_name in [c.name for c in self.client.list_collections()]:
                self.collection = self.client.get_collection(name=self.collection_name)
                print(f"Collection '{self.collection_name}' loaded.")
            else:
                self.collection = self.client.create_collection(
                    name=self.collection_name,
                    metadata={"Description": "PDF embedding for RAG."}
                )
                print(f"Collection '{self.collection_name}' created.")

            print("Existing Documents in collection: ", self.collection.count())

        except Exception as e:
            print(f"Error initializing vector store: {e}")
            self.collection = None

    
    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents and embeddings must match.")
        
        print(f"Adding {len(documents)} documents to the vector store...")

        ids=[]
        metadatas=[]
        documents_texts=[]
        embeddings_list=[]

        for i,(doc, emb) in enumerate(zip(documents, embeddings)):
            # Generate a unique ID for each document
            doc_id=f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            #Make metadata
            metadata=dict(doc.metadata)
            metadata['doc_index']= i
            metadata['content_length']= len(doc.page_content)
            metadatas.append(metadata)

            # Document Context
            documents_texts.append(doc.page_content)

            # Embeddings
            embeddings_list.append(emb.tolist())
        
        try:
            self.collection.add(
                ids=ids,
                metadatas=metadatas,
                documents=documents_texts,
                embeddings=embeddings_list
            )
            print(f"Added {len(documents)} documents successfully.")
            print("Total Documents in collection after addition: ", self.collection.count())
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

        

In [27]:
vs=VectorStore()

Initializing ChromaDB client...
ChromaDB client initialized.
Collection 'Doc_data' created.
Existing Documents in collection:  0


In [29]:
#Convert the text to embeddings

texts = [doc.page_content for doc in all_subren_docs]

#Generate embeddings
embeddings = eM.generate_embeddings(texts)


import numpy as np
import math

MAX_BATCH_SIZE = 5000  # safe number below 5461

def add_documents_in_batches(vector_store, documents, embeddings):
    total_docs = len(documents)
    num_batches = math.ceil(total_docs / MAX_BATCH_SIZE)
    print(f"Adding {total_docs} documents in {num_batches} batches...")

    for i in range(num_batches):
        start = i * MAX_BATCH_SIZE
        end = min((i + 1) * MAX_BATCH_SIZE, total_docs)

        batch_docs = documents[start:end]
        batch_embeddings = embeddings[start:end]

        vector_store.add_documents(batch_docs, batch_embeddings)
        print(f"Batch {i+1}/{num_batches} added: {len(batch_docs)} documents")

#Store in vector DB
add_documents_in_batches(vs, all_subren_docs, embeddings)




# vs.add_documents(all_subren_docs, embeddings)

Batches: 100%|██████████| 1006/1006 [06:23<00:00,  2.62it/s]


Generated embeddings for 32180 texts.
Embedding shape: (32180, 384)
Adding 32180 documents in 7 batches...
Adding 5000 documents to the vector store...
Added 5000 documents successfully.
Total Documents in collection after addition:  5000
Batch 1/7 added: 5000 documents
Adding 5000 documents to the vector store...
Added 5000 documents successfully.
Total Documents in collection after addition:  10000
Batch 2/7 added: 5000 documents
Adding 5000 documents to the vector store...
Added 5000 documents successfully.
Total Documents in collection after addition:  15000
Batch 3/7 added: 5000 documents
Adding 5000 documents to the vector store...
Added 5000 documents successfully.
Total Documents in collection after addition:  20000
Batch 4/7 added: 5000 documents
Adding 5000 documents to the vector store...
Added 5000 documents successfully.
Total Documents in collection after addition:  25000
Batch 5/7 added: 5000 documents
Adding 5000 documents to the vector store...
Added 5000 documents suc

### Retriver Pipeline

In [33]:
class RagRetriever:

    def __init__(self, vector_store : vs, embedding_manager : eM):
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k : int =5, score_threshold : float =0.0) -> List[Dict[str, Any]]:
        #Query : Search querry , top_k : number of similar documents to retrieve, score_threshold : minimum similarity score

        print(f"Generating embedding for query: {query}")
        print(f"Top K: {top_k}, Score Threshold: {score_threshold}")

        #Generate embedding for the query
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        #Perform similarity search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k,
            )
            print(f"Retrieved {len(results['ids'][0])} documents from vector store.")

            #Filter based on score threshold
            retrieved_docs = []

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                ids = results['ids'][0]
                distances = results['distances'][0]

                for i, (doc_id, doc, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    similarity_score = 1 - distance  # Convert distance to similarity score
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            "id": doc_id,
                            "document": doc,
                            "metadata": metadata,
                            "similarity_score": similarity_score,
                            "distance": distance,
                            "rank": i + 1
                        })
                        print(f"Doc ID: {doc_id}, Similarity Score: {similarity_score:.4f}")
                    else:
                        print(f"Doc ID: {doc_id} filtered out due to low similarity score: {similarity_score:.4f}")

                print("Total Retrieved Documents after filtering: ", len(retrieved_docs))
            else:
                print("No documents retrieved.")
            return retrieved_docs
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

            


In [34]:
rag_retriever = RagRetriever(vector_store=vs, embedding_manager=eM)


In [50]:
rag_retriever.retrieve("What do you know about SUBREN-415 ?")

Generating embedding for query: What do you know about SUBREN-415 ?
Top K: 5, Score Threshold: 0.0


Batches: 100%|██████████| 1/1 [00:00<00:00, 37.14it/s]

Generated embeddings for 1 texts.
Embedding shape: (1, 384)
Retrieved 5 documents from vector store.
Doc ID: doc_691a145b_429, Similarity Score: 0.1437
Doc ID: doc_405d52ac_3847, Similarity Score: 0.0805
Doc ID: doc_3e4333f9_3275, Similarity Score: 0.0516
Doc ID: doc_7766f94a_3867, Similarity Score: 0.0512
Doc ID: doc_5ae7d362_4521, Similarity Score: 0.0348
Total Retrieved Documents after filtering:  5


[{'id': 'doc_691a145b_429',
  'document': '[SUBREN-927]\xa0Part 3: Mapping Document for Sabrix Interface Created: 13/Jul/22 \nUpdated: 01/Aug/22 \xa0Resolved: 29/Jul/22 \nStatus:\nClosed\nProject:\nSubscription & Renewals\nComponent/s:\nNone \nAffects Version/s:\nNone \nFix Version/s:\nNone \nType: \nStory \nPriority: \nNone \nReporter: \nYolanda Balague \nAssignee: \nJesse Smith \nResolution: \nDone \nVotes: \n0 \nLabels: \nA&E, NoQA \nRemaining \nEstimate:\n2h \nTime Spent:\nNot Specified \nOriginal Estimate: 2h \nIssue Links: \nCloners\nclones SUBREN-\n840\n \n  \nPart 2: \nMapping \nDocument \nfor Sabrix \nI... \nClosed \nRelates\nrelates \nto \nSUBREN-\n318\n \n  \nSF to Send \nAll \nInformation \non IC \nInvo... \nClosed \nrelates \nto \nSUBREN-\n319\n \n  \nSPIKE: SF \nto Send All \nInformation \nfor... \nClosed \nrelates \nto \nSUBREN-\n320\n \n  \nSF to Send \nAll Data on \nService \nClosed',
  'metadata': {'content_length': 786,
   'file_type': 'pdf',
   'file_path': '../data

## LLM

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()  # Make sure environment variables are loaded

def answer_with_rag(query: str, retriever, top_k: int = 3) -> str:
    """
    Retrieve relevant documents from local vector store and generate
    an answer using Azure OpenAI LLM.
    """
    # 1️⃣ Retrieve top_k documents from local vector store
    retrieved_docs = retriever.retrieve(query, top_k=top_k, score_threshold=0.1)

    if not retrieved_docs:
        return "No relevant documents found in the vector store."

    # 2️⃣ Prepare context string
    context = "\n\n".join([doc["document"] for doc in retrieved_docs])
    # print(context)
    # 3️⃣ Prepare prompt for LLM
    system_message = f"""
You are a QA Engineer at Keysight. Answer the user's question based on the context provided below.


Context:
{context}
"""

    user_message = f"User Question: {query}"


    client = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OAI_KEY"),
        api_version="2024-05-01-preview"
    )

   
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]

 
    try:
        response = client.chat.completions.create(
            model=os.getenv("AZURE_OAI_DEPLOYMENT"),  # e.g., gpt-4o
            max_tokens=10000,
            messages=messages
        )

        answer = response.choices[0].message.content
        print("Answer Generated:\n", answer)
        # Optional: print annotations if available
        if hasattr(response.choices[0].message, "annotations"):
            print("\nCitations / Annotations:")
            for a in response.choices[0].message.annotations:
                print(a)

        return answer

    except Exception as e:
        return f"Error generating answer: {e}"


In [55]:
ans=answer_with_rag("What do you know about Voscal Deal Creation - Discount Aggregation Logic ?", rag_retriever)
ans

Generating embedding for query: What do you know about Voscal Deal Creation - Discount Aggregation Logic ?
Top K: 3, Score Threshold: 0.0


Batches: 100%|██████████| 1/1 [00:00<00:00, 27.64it/s]

Generated embeddings for 1 texts.
Embedding shape: (1, 384)
Retrieved 3 documents from vector store.
Doc ID: doc_e54e05dc_2907, Similarity Score: 0.2512
Doc ID: doc_d66ad6d2_2901, Similarity Score: 0.2475
Doc ID: doc_8ceb7d74_487, Similarity Score: 0.2183
Total Retrieved Documents after filtering:  3


Answer Generated:
 From the provided context regarding the Voscal Deal Creation and Discount Aggregation Logic, here is what I understand:

1. **Voscal Pricing and Logic**:
   - In Voscal deals, the prices are set to NPA (No Price Agreement) for the specified Voscal quote.
   - These deals often include items priced at $0 because they are part of the Voscal pricing structure. This is not considered a discount but a pricing mechanism.
   - The "Voscal Planner" updates the NPA price, although they do not have a persona in the tool.
   - Entitlement-only items are set to override the price to $0 at the line level. These will not flow to the EBS order.
   - Non-discountable logic would need to be applied, ensuring items are not aggregated or discounted improperly in the purchase agreement.

2. **Manual Process**:
   - The current process involves manual steps for Voscal deals, such as manual entry of CFDs.
   - RSS/CSR manually sets entitlement-only lines, and the CFD team is engaged for p

'From the provided context regarding the Voscal Deal Creation and Discount Aggregation Logic, here is what I understand:\n\n1. **Voscal Pricing and Logic**:\n   - In Voscal deals, the prices are set to NPA (No Price Agreement) for the specified Voscal quote.\n   - These deals often include items priced at $0 because they are part of the Voscal pricing structure. This is not considered a discount but a pricing mechanism.\n   - The "Voscal Planner" updates the NPA price, although they do not have a persona in the tool.\n   - Entitlement-only items are set to override the price to $0 at the line level. These will not flow to the EBS order.\n   - Non-discountable logic would need to be applied, ensuring items are not aggregated or discounted improperly in the purchase agreement.\n\n2. **Manual Process**:\n   - The current process involves manual steps for Voscal deals, such as manual entry of CFDs.\n   - RSS/CSR manually sets entitlement-only lines, and the CFD team is engaged for process 

In [60]:
ans=answer_with_rag("What are the top Comments on Jira [SUBREN-2377] 6- Orders erroring due to duplicate customer hold Created: ?", rag_retriever)

print("Final Answer:\n", ans)


Generating embedding for query: What are the top Comments on Jira [SUBREN-2377] 6- Orders erroring due to duplicate customer hold Created: ?
Top K: 3, Score Threshold: 0.0


Batches: 100%|██████████| 1/1 [00:00<00:00, 19.62it/s]

Generated embeddings for 1 texts.
Embedding shape: (1, 384)
Retrieved 3 documents from vector store.
Doc ID: doc_d466a197_3727, Similarity Score: 0.2607
Doc ID: doc_17e14875_3981, Similarity Score: 0.1991
Doc ID: doc_75feeaee_233, Similarity Score: 0.1833
Total Retrieved Documents after filtering:  3


Answer Generated:
 The top comments on Jira [SUBREN-2377] are:

1. **Comment by Yolanda Balague [22/Jan/24]:**  
   * "as per comments"  
   * "Closed, according to IT after mid-November there have not been any cases and agreed with Tarun that if he sees something new then we open a new bug."

2. **Comment by Virginia Garcia-Medina [19/Oct/23]:**  
   * "Both orders are activated."

Citations / Annotations:
Final Answer:
 The top comments on Jira [SUBREN-2377] are:

1. **Comment by Yolanda Balague [22/Jan/24]:**  
   * "as per comments"  
   * "Closed, according to IT after mid-November there have not been any cases and agreed with Tarun that if he sees something new then we open a new bug."

2. **Comment by Virginia Garcia-Medina [19/Oct/23]:**  
   * "Both orders are activated."


In [61]:
ans=answer_with_rag("1-Discount AMT - calculated against Prorated Price. It should be calculated against Monthly price WHAT ARE THE TOP COMMENTS ON THIS JIRA  ?", rag_retriever)

print("Final Answer:\n", ans)


Generating embedding for query: 1-Discount AMT - calculated against Prorated Price. It should be calculated against Monthly price WHAT ARE THE TOP COMMENTS ON THIS JIRA  ?
Top K: 3, Score Threshold: 0.0


Batches: 100%|██████████| 1/1 [00:00<00:00, 36.24it/s]

Generated embeddings for 1 texts.
Embedding shape: (1, 384)
Retrieved 3 documents from vector store.
Doc ID: doc_7571c5ad_2237, Similarity Score: 0.3050
Doc ID: doc_cee571b3_2245, Similarity Score: 0.2353
Doc ID: doc_e4b6d49c_2639, Similarity Score: 0.2335
Total Retrieved Documents after filtering:  3


Answer Generated:
 The top comments on this Jira are:

1. **Kim Hamid [05/Feb/24]**: Mentioned that the test is certified and ready to deploy by February 9. Also requested Ankit Kapoor to add story points to this ticket.

2. **Shubham Saini [31/Jan/24]**: Confirmed that this is working fine as per the comments and is test-verified. Tagged Nitin Job for confirmation.

3. **Nitin Job [31/Jan/24]**: Requested Shubham Saini to check the comments above and confirm if everything is good.

4. **Kevin York [31/Jan/24]**: Explained that for a Korean quote, the system rounds the monthly net price to the nearest thousand, which impacts the effective discount percentage, but clarified that the calculations are correct.

Citations / Annotations:
Final Answer:
 The top comments on this Jira are:

1. **Kim Hamid [05/Feb/24]**: Mentioned that the test is certified and ready to deploy by February 9. Also requested Ankit Kapoor to add story points to this ticket.

2. **Shubham Saini [31/Jan/24]**: Confi

## Advanced Rag

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()  # Make sure environment variables are loaded

def rag_adv(query: str, retriever, top_k: int = 5, min_score: float = 0.2, return_context: bool = False) -> str:
    """
    Retrieve relevant documents from local vector store and generate
    an answer using Azure OpenAI LLM.
    """

    retrieved_docs = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)

    if not retrieved_docs:
        return {"answer": "No relevant documents found in the vector store.", 'sources': [], 'confidence': 0.0, 'context': ''}

    context = "\n\n".join([doc["document"] for doc in retrieved_docs])
    sources =[{
        "source": doc["metadata"].get("source_file", doc["metadata"].get("source",'Unknown')),
        'page' :doc["metadata"].get("page", 'N/A'),
        'score' : doc["similarity_score"],
        'preview': doc["document"][:300] +"...",
    }for doc in retrieved_docs]

    confidence = max([doc["similarity_score"] for doc in retrieved_docs]) if retrieved_docs else 0.0

    system_message = f"""
    You are a QA Engineer at Keysight where you need to answer the user's question based on the jira context provided below.

Context:
{context} 

Question: {query}

"""

    user_message = f"User Question: {query}"


    client = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OAI_KEY"),
        api_version="2024-05-01-preview"
    )

   
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]

 
    try:
        response = client.chat.completions.create(
            model=os.getenv("AZURE_OAI_DEPLOYMENT"),  # e.g., gpt-4o
            max_tokens=10000,
            messages=messages
        )

        answer = response.choices[0].message.content

        output = {
            "answer": answer,
            "sources": sources,
            "confidence": confidence,
        }

        # print("Answer Generated:\n", answer)
        # Optional: print annotations if available
        # if hasattr(response.choices[0].message, "annotations"):
        #     print("\nCitations / Annotations:")
        #     for a in response.choices[0].message.annotations:
        #         print(a)
        if return_context:
            output['context'] = context

        return output

    except Exception as e:
        return f"Error generating answer: {e}"
